In [ ]:
# default_exp simulators.chp

# CHP Simulator

> Implementation of the Aarenson/Gottesman CHP stabilizer simulator

This simulator updates the stabilizer generators of an n-qubit state (initially in $|00..0 \rangle$) after each gate application. It implements the fundamental gates CNOT, Hadamard and Phase (S-gate) from which all other gates of the Clifford group can be generated. The foundation of this simulator follows the Gottesman/Aarenson paper closely and was originally written by Craig Gidney. The implementation is well tested and comes with a lot of nice tests written by the original author.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from qsam.simulators.mixins import CircuitRunnerMixin
from typing import Union, Any
import numpy as np
import random

In [ ]:
#hide
#export
def pauli_product_phase(x1: bool, z1: bool, x2: bool, z2: bool) -> int:
    """Determines the power of i in the product of two Paulis.
    For example, X*Y = iZ and so this method would return +1 for X and Y.
    The input Paulis are encoded into the following form:
        x z | Pauli
        ----+-------
        0 0 | I
        1 0 | X
        1 1 | Y
        0 1 | Z
    """
    # Analyze by case over first gate.

    if x1 and z1:  # Y gate.
        # No phase for YI = Y
        # -1 phase for YX = -iZ
        # No phase for YY = I
        # +1 phase for YZ = +iX
        return int(z2) - int(x2)

    if x1:  # X gate.
        # No phase for XI = X
        # No phase for XX = I
        # +1 phase for XY = iZ
        # -1 phase for XZ = -iY
        return z2 and 2*int(x2) - 1

    if z1:  # Z gate.
        # No phase for ZI = Z
        # +1 phase for ZX = -iY
        # -1 phase for ZY = iX
        # No phase for ZZ = I
        return x2 and 1 - 2*int(z2)

    # Identity gate.
    return 0

In [ ]:
#hide
#export
class MeasureResult:
    """A measurement's output and whether it was random or not."""

    def __init__(self, value: bool, determined: bool):
        self.value = bool(value)
        self.determined = bool(determined)

    def __bool__(self):
        return self.value

    def __eq__(self, other):
        if isinstance(other, (bool, int)):
            return self.value == other
        if isinstance(other, MeasureResult):
            return self.value == other.value and self.determined == other.determined
        return NotImplemented

    def __str__(self):
        return '{} ({})'.format(self.value,
                                ['random', 'determined'][self.determined])

    def __repr__(self):
        return 'MeasureResult(value={!r}, determined={!r})'.format(
            self.value,
            self.determined)

In [ ]:
#export
class ChpSimulator(CircuitRunnerMixin):
    """The bare minimum needed for the CHP simulation.
    
    Reference:
        "Improved Simulation of Stabilizer Circuits"
        Scott Aaronson and Daniel Gottesman
        https://arxiv.org/abs/quant-ph/0406196
        
    Original author:
        Craig Gidney
        https://github.com/Strilanc/python-chp-stabilizer-simulator
    """
    
    def __init__(self, num_qubits):
        self._n = num_qubits
        self._table = np.eye(2 * num_qubits + 1, dtype=bool)
        self._x = self._table[:, :self._n]
        self._z = self._table[:, self._n:-1]
        self._r = self._table[:, -1]
        
    def init(self, qubit: int) -> None:
        """Initialize to |0>"""
        m = self.measure(qubit)
        if m == 1: 
            self.X(qubit)
        
    def I(self, qubit: int) -> None:
        """Identity gate"""
        pass
        
    def Z(self, qubit: int) -> None:
        """Z gate"""
        self.S(qubit)
        self.S(qubit)
        
    def X(self, qubit: int) -> None:
        """X gate"""
        self.H(qubit)
        self.Z(qubit)
        self.H(qubit)
        
    def Y(self, qubit: int) -> None:
        """Y gate"""
        self.S(qubit)
        self.S(qubit)
        self.S(qubit)
        self.X(qubit)
        self.S(qubit)
        
    def H(self, qubit:int) -> None:
        """H gate"""
        self.hadamard(qubit)
        
    def CNOT(self, control: int, target: int) -> None:
        """CNOT gate"""
        self.cnot(control, target)
        
    def S(self, qubit: int) -> None:
        """Phase gate"""
        self.phase(qubit)
        
    def Q(self, qubit: int) -> None:
        """Q = sqrt(X) = HSH"""
        self.H(qubit)
        self.S(qubit)
        self.H(qubit)
    
    def Qd(self, qubit: int) -> None:
        """Q^(dagger) = Q^3"""
        self.Q(qubit)
        self.Q(qubit)
        self.Q(qubit)
        
    def Sd(self, qubit: int) -> None:
        """S^(dagger) = S^3"""
        self.S(qubit)
        self.S(qubit)
        self.S(qubit)
    
    def R(self, qubit: int) -> None:
        """R = sqrt(XZ) = SQS^(dagger)"""
        self.Sd(qubit)
        self.Q(qubit)
        self.S(qubit)
        
    def Rd(self, qubit: int) -> None:
        """R^(dagger) = R^3"""
        self.R(qubit)
        self.R(qubit)
        self.R(qubit)
        
    def MSd(self, qubitA: int, qubitB: int) -> None:
        """Molmer-Sorensen gate: -pi/2 XX rotation
        Ref.: Fig. 6 of https://arxiv.org/pdf/2111.12654.pdf"""
        self.Rd(qubitA)
        self.CNOT(qubitA, qubitB)
        self.R(qubitA)
        self.Qd(qubitA)
        self.Qd(qubitB)
        
    def hadamard(self, qubit: int) -> None:
        """Applies a Hadamard gate to a qubit.
        
        Args:
            qubit: The qubit to apply the H gate to.
        """
        self._r[:] ^= self._x[:, qubit] & self._z[:, qubit]
        # Perform a XOR-swap
        self._x[:, qubit] ^= self._z[:, qubit]
        self._z[:, qubit] ^= self._x[:, qubit]
        self._x[:, qubit] ^= self._z[:, qubit]
        
    def phase(self, qubit: int) -> None:
        """Applies an S gate to a qubit.
        
        Args:
            qubit: The qubit to apply the S gate to.
        """
        self._r[:] ^= self._x[:, qubit] & self._z[:, qubit]
        self._z[:, qubit] ^= self._x[:, qubit]
        
    def cnot(self, control: int, target: int) -> None:
        """Applies a CNOT gate between two qubits.
        
        Args:
            control: The control qubit of the CNOT.
            target: The target qubit of the CNOT.
        """
        self._r[:] ^= self._x[:, control] & self._z[:, target] & (
                self._x[:, target] ^ self._z[:, control] ^ True)
        self._x[:, target] ^= self._x[:, control]
        self._z[:, control] ^= self._z[:, target]
        
    def measure(self,
                qubit: int,
                *,
                bias: Union[float, int, bool] = 0.5) -> 'MeasureResult':
        """Computational basis (Z basis) measurement.
        
        Args:
            qubit: The index of the qubit to measure.
            bias: When the measurement result is random, this is the probability
                of getting a True result value instead of False. Mostly useful
                for making reproducible unit tests.
                
        Returns:
            A MeasurementResult instance whose `value` attribute is the outcome
            of the measurement and whose `determined` attribute indicates
            whether the outcome was deterministic or random.
        """
        n = self._n
        for p in range(n):
            if self._x[p+n, qubit]:
                return self._measure_random(qubit, p, bias)
        return self._measure_determined(qubit)
    
    def _measure_random(self,
                        a: int,
                        p: int,
                        bias: Union[float, int, bool]) -> 'MeasureResult':
        n = self._n
        assert self._x[p+n, a]
        self._table[p, :] = self._table[p + n, :]
        self._table[p + n, :] = 0
        self._z[p + n, a] = 1
        self._r[p + n] = random.random() < bias

        for i in range(2*n):
            if self._x[i, a] and i != p and i != p + n:
                self._row_mult(i, p)
        return MeasureResult(value=self._r[p + n], determined=False)
    
    def _measure_determined(self, a: int) -> 'MeasureResult':
        n = self._n
        self._table[-1, :] = 0
        for i in range(n):
            if self._x[i, a]:
                self._row_mult(-1, i + n)
        return MeasureResult(value=self._r[-1], determined=True)
    
    def _row_product_sign(self, i: int, k: int) -> bool:
        """Determines the sign of two rows' Pauli Products."""
        pauli_phases = sum(
            pauli_product_phase(self._x[i, j],
                                self._z[i, j],
                                self._x[k, j],
                                self._z[k, j])
            for j in range(self._n)
        )
        assert not pauli_phases & 1, (
            "Expected commuting rows but got {}, {} from \n{}".format(
                i, k, self))
        p = (pauli_phases >> 1) & 1
        return bool(self._r[i] ^ self._r[k] ^ p)
    
    def _row_mult(self, i: int, k: int) -> None:
        """Multiplies row k's Paulis into row i's Paulis."""
        self._r[i] = self._row_product_sign(i, k)
        self._x[i, :self._n] ^= self._x[k, :self._n]
        self._z[i, :self._n] ^= self._z[k, :self._n]
        
    def __str__(self):
        """Represents the state as a list of Pauli products.
        Each Pauli product is what the X or Z observable of a qubit at the
        current time was equal to at time zero (after accounting for
        measurements).
        """

        def _cell(row: int, col: int) -> str:
            k = int(self._x[row, col]) + 2 * int(self._z[row, col])
            return ['.', 'X', 'Z', 'Y'][k]

        def _row(row: int) -> str:
            result = '-' if self._r[row] else '+'
            for col in range(self._n):
                result += str(_cell(row, col))
            return result

        z_obs = [_row(row) for row in range(self._n)]
        sep = ['-' * (self._n + 1)]
        x_obs = [_row(row) for row in range(self._n, 2 * self._n)]
        return '\n'.join(z_obs + sep + x_obs)

    def _repr_pretty_(self, p: Any, cycle: bool) -> None:
        p.text(str(self))

Following are tests of the implementation.

In [ ]:
from qsam.circuit import Circuit

circuit = Circuit([{'init': [0,1,2,3]}, {'measure': [0,1,2,3]}])
f_circuit = Circuit([{}, {'X': [0,3]}])

s = ChpSimulator(num_qubits=4)
s.run(circuit, f_circuit)

'1001'

In [ ]:
#hide
import collections
from typing import Set
import itertools

Test the phase of a Pauli product $\sigma_i \sigma_j$. Output is in powers of i, i.e. $i^k=\{-i,1,i\}$ for $k=\{-1,0,1\}$:

In [ ]:
def test_pauli_product_phase():
    paulis = [
        [False, False],  # I
        [True, False],  # X
        [True, True],  # Y
        [False, True],  # Z
    ]
    expected = [
        [0, 0, 0, 0],
        [0, 0, 1, -1],
        [0, -1, 0, 1],
        [0, 1, -1, 0],
    ]
    assert pauli_product_phase(x1=True, z1=False, x2=True, z2=True) == 1;
    for i in range(4):
        for j in range(4):
            assert pauli_product_phase(*paulis[i], *paulis[j]) == expected[i][j]

test_pauli_product_phase()

Test that one qubit is initialized in $|0\rangle$:

In [ ]:
def test_identity():
    s = ChpSimulator(num_qubits=1)
    assert s.measure(0) == MeasureResult(value=False, determined=True)
    
test_identity()

Test measure $X|0\rangle$ in $Z$ always determined (-1):

In [ ]:
def test_bit_flip():
    s = ChpSimulator(num_qubits=1)
    s.hadamard(0)
    s.phase(0)
    s.phase(0)
    s.hadamard(0)
    assert s.measure(0) == MeasureResult(value=True, determined=True)
    
test_bit_flip()

Test that two qubits are initialized in $|00\rangle$:

In [ ]:
def test_identity_2():
    s = ChpSimulator(num_qubits=2)
    assert s.measure(0) == MeasureResult(value=False, determined=True)
    assert s.measure(1) == MeasureResult(value=False, determined=True)
    
test_identity_2()

Test measure $X_AX_B|0_A0_B\rangle$ in $Z_AZ_B$ always determined (-1):

In [ ]:
def test_bit_flip_2():
    s = ChpSimulator(num_qubits=2)
    s.hadamard(0)
    s.phase(0)
    s.phase(0)
    s.hadamard(0)
    assert s.measure(0) == MeasureResult(value=True, determined=True)
    assert s.measure(1) == MeasureResult(value=False, determined=True)
    
test_bit_flip_2()

Test measure $\frac{1}{\sqrt{2}}(|0_A0_B\rangle+|1_A1_B\rangle)$ in $Z_B$ always undetermined (i.e. random) but measure $Z_A$ on post-measurement state must give same result and be determined, as the first measurement collapsed state to either $|0_A\rangle$ or $|1_A\rangle$:

In [ ]:
def test_epr():
    s = ChpSimulator(num_qubits=2)
    s.hadamard(0)
    s.cnot(0, 1)
    v1 = s.measure(0)
    assert not v1.determined
    v2 = s.measure(1)
    assert v2.determined
    assert v1.value == v2.value
    
test_epr()

Test measure $|\psi\rangle=\frac{1}{2}(|00\rangle+|11\rangle+i|01\rangle+i|10\rangle)$ in $Z_B$ always undetermined. If result is +1 (i.e. $|0_B\rangle$) applying one phase, if result is -1 applying 3 phases brings state to $|+\rangle$, s.t. $H|+\rangle=|0\rangle$ and subseqeunt measurement should always be determined:

In [ ]:
def test_phase_kickback_consume_s_state():
    s = ChpSimulator(num_qubits=2)
    s.hadamard(1)
    s.phase(1)
    s.hadamard(0)
    s.cnot(0, 1)
    v1 = s.measure(1)
    assert not v1.determined
    if v1:
        s.phase(0)
        s.phase(0)
    s.phase(0)
    s.hadamard(0)
    assert s.measure(0) == MeasureResult(value=True, determined=True)

test_phase_kickback_consume_s_state()

Test input $|\psi\rangle=\frac{1}{2}(|00\rangle+|10\rangle+i|01\rangle+i|11\rangle)$, after phase kickback $|\psi^{\prime}\rangle=\frac{1}{2}(|00\rangle+i|10\rangle+i|01\rangle-|11\rangle)$, before measurement $\psi^{\prime\prime}=|1\rangle \otimes |+^{\prime}\rangle$ with $|\pm^{\prime}\rangle$ being the $Y$-Pauli eigenstates. Thus measure $Z_A$ is determined with post-measurement state $|+^{\prime}$ which becomes $H_BS_B|+^{\prime}=|1_B\rangle$ s.t. measuring $Z_B$ is also determined:

In [ ]:
def test_phase_kickback_preserve_s_state():
    s = ChpSimulator(num_qubits=2)

    # Prepare S state.
    s.hadamard(1)
    s.phase(1)

    # Prepare test input.
    s.hadamard(0)

    # Kickback.
    s.cnot(0, 1)
    s.hadamard(1)
    s.cnot(0, 1)
    s.hadamard(1)

    # Check.
    s.phase(0)
    s.hadamard(0)
    assert s.measure(0) == MeasureResult(value=True, determined=True)
    s.phase(1)
    s.hadamard(1)
    assert s.measure(1) == MeasureResult(value=True, determined=True)

test_phase_kickback_preserve_s_state()

Test if stabilizer gens and destabilizer gens are transformed correctly. Stabilizers and destabilizers together generate the Pauli group $P_n$ for $n$ qubits at all times. The stabilizers and destabilizers for an initial state of $|0\rangle^{\otimes n}$ are $\{Z_i: i=1,..,n\}$ and $\{X_i: i=1,..,n\}$, respectively. The `__repr__` string of the simulator outputs the current destabilizers followed by the stabilizers. Thus, we'd expect $ZII\rightarrow XIX$, $IZI\rightarrow IXX$, $IIZ\rightarrow YYZ$ for the stabilizer generators and $XII\rightarrow -YII$, $IXI\rightarrow -IYI$ and $IIX\rightarrow IIX$ for the destabilizer generators in the first assert, and so on:

In [ ]:
def test_kickback_vs_stabilizer():
    sim = ChpSimulator(num_qubits=3)
    sim.hadamard(2)
    sim.cnot(2, 0)
    sim.cnot(2, 1)
    sim.phase(0)
    sim.phase(1)
    sim.hadamard(0)
    sim.hadamard(1)
    sim.hadamard(2)
    assert str(sim).strip() == """
-YII
-IYI
+IIX
----
+XIX
+IXX
+YYZ
    """.strip().replace('I', '.')
    v0 = sim.measure(0, bias=0)
    assert str(sim).strip() == """
+XIX
-IYI
+IIX
----
+ZII
+IXX
+ZYY
    """.strip().replace('I', '.')
    v1 = sim.measure(1, bias=0)
    assert str(sim).strip() == """
+XIX
+IXX
+IIX
----
+ZII
+IZI
-ZZZ
    """.strip().replace('I', '.')
    v2 = sim.measure(2, bias=0)
    assert str(sim).strip() == """
+XIX
+IXX
+IIX
----
+ZII
+IZI
-ZZZ
    """.strip().replace('I', '.')
    assert v0 == MeasureResult(value=False, determined=False)
    assert v1 == MeasureResult(value=False, determined=False)
    assert v2 == MeasureResult(value=True, determined=True)
    
test_kickback_vs_stabilizer()

The following 3 tests accomplish magic state distillation from 9 (5) qubits to a single one and probe different properties of this process:

In [ ]:
def test_s_state_distillation_low_depth():
    for _ in range(100):
        sim = ChpSimulator(num_qubits=9)

        stabilizers = [
            (0, 1, 2, 3),
            (0, 1, 4, 5),
            (0, 2, 4, 6),
            (1, 2, 4, 7)
        ]
        checks = [
            {'s': [0], 'q': stabilizers[0]},
            {'s': [1], 'q': stabilizers[1]},
            {'s': [2], 'q': stabilizers[2]},
        ]

        stabilizer_measurements = []
        anc = 8
        for stabilizer in stabilizers:
            sim.hadamard(anc)
            for k in stabilizer:
                sim.cnot(anc, k)
            sim.hadamard(anc)
            v = sim.measure(anc)
            assert not v.determined
            if v.value:
                sim.hadamard(anc)
                sim.phase(anc)
                sim.phase(anc)
                sim.hadamard(anc)
            stabilizer_measurements.append(v)

        qubit_measurements = []
        for k in range(7):
            sim.phase(k)
            sim.hadamard(k)
            qubit_measurements.append(sim.measure(k))

        if sum([int(e.value) for e in stabilizer_measurements + qubit_measurements]) & 1:
            sim.phase(7)
            sim.phase(7)

        sim.phase(7)
        sim.hadamard(7)
        r = sim.measure(7)

        assert r == MeasureResult(value=False, determined=True)

        for c in checks:
            rvs = [int(stabilizer_measurements[k].value) for k in c['s']]
            rms = [int(qubit_measurements[k].value) for k in c['q']]
            assert sum(rvs + rms) & 1 == 0
            
test_s_state_distillation_low_depth()

In [ ]:
def test_s_state_distillation_low_space():
    for _ in range(100):
        sim = ChpSimulator(num_qubits=5)

        phasors = [
            (0,),
            (1,),
            (2,),
            (0, 1, 2),
            (0, 1, 3),
            (0, 2, 3),
            (1, 2, 3),
        ]

        anc = 4
        for phasor in phasors:
            sim.hadamard(anc)
            for k in phasor:
                sim.cnot(anc, k)
            sim.hadamard(anc)
            sim.phase(anc)
            sim.hadamard(anc)
            v = sim.measure(anc)
            assert not v.determined
            if v.value:
                for k in phasor + (anc,):
                    sim.hadamard(k)
                    sim.phase(k)
                    sim.phase(k)
                    sim.hadamard(k)

        for k in range(3):
            assert sim.measure(k) == MeasureResult(value=False, determined=True)
        sim.phase(3)
        sim.hadamard(3)
        assert sim.measure(3) == MeasureResult(value=True, determined=True)
        
test_s_state_distillation_low_space()

In [ ]:
def test_count_s_state_distillation_failure_cases():
    def distill(errors: Set[int]) -> str:
        sim = ChpSimulator(num_qubits=5)

        phasors = [
            (0,),
            (1,),
            (2,),
            (0, 1, 2),
            (0, 1, 3),
            (0, 2, 3),
            (1, 2, 3),
        ]

        anc = 4
        for e, phasor in enumerate(phasors):
            for k in phasor:
                sim.hadamard(anc)
                sim.cnot(anc, k)
                sim.hadamard(anc)
            sim.phase(anc)

            if e in errors:
                sim.phase(anc)
                sim.phase(anc)

            sim.hadamard(anc)
            v = sim.measure(anc)
            if v.value:
                sim.hadamard(anc)
                sim.phase(anc)
                sim.phase(anc)
                sim.hadamard(anc)
            assert not v.determined
            if v.value:
                for k in phasor:
                    sim.hadamard(k)
                    sim.phase(k)
                    sim.phase(k)
                    sim.hadamard(k)

        sim.phase(3)
        sim.phase(3)
        sim.phase(3)
        sim.hadamard(3)
        result = sim.measure(3)
        sim.hadamard(3)
        sim.phase(3)
        checks = [sim.measure(k) for k in range(3)]
        assert result.determined
        assert all(e.determined for e in checks)
        good_result = result.value is False
        checks_passed = not any(e.value for e in checks)
        if checks_passed:
            if good_result:
                return 'good'
            else:
                return 'ERROR'
        else:
            if good_result:
                return 'victim'
            else:
                return 'caught'

    def classify(errs) -> collections.Counter:
        result = collections.Counter()
        for err in errs:
            result[distill(err)] += 1
        return result

    nones = list(itertools.combinations(range(7), 0))
    singles = list(itertools.combinations(range(7), 1))
    doubles = list(itertools.combinations(range(7), 2))
    triples = list(itertools.combinations(range(7), 3))

    assert classify(nones) == {'good': 1}
    assert classify(singles) == {'caught': 3, 'victim': 4}
    assert classify(doubles) == {'caught': 12, 'victim': 9}
    assert classify(triples) == {'caught': 12, 'victim': 16, 'ERROR': 7}
    
test_count_s_state_distillation_failure_cases()